# EDA TyDi QA
Notebook untuk memahami dataset TyDi QA

In [14]:
from collections import Counter
import pandas as pd
from datasets import load_dataset
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [9]:
dataset = load_dataset("khalidalt/tydiqa-goldp", 'indonesian', trust_remote_code=True)

# Ambil kolom 'question_text' dari dataset train
questions = dataset['train']['question_text']

# Ambil kata pertama dari setiap pertanyaan
first_words = [question.lower().split()[0] if question.strip() else None for question in questions]

# Hitung jumlah kemunculan tiap kata pertama
first_word_counts = Counter(first_words)

# Konversi hasil ke DataFrame untuk tampilan lebih mudah
first_word_df = pd.DataFrame(first_word_counts.items(), columns=["First Word", "Count"]).sort_values(by="Count", ascending=False)

# Tampilkan hasil
first_word_df.head(10)

,First Word,Count
5,apakah,953
2,kapan,919
6,apa,851
4,siapakah,528
7,berapakah,406
8,berapa,391
1,siapa,379
3,dimana,255
0,dimanakah,245
9,kapankah,238


## Menggunakan meta-llama_Llama-3.2-1B-Instruct-50_results.json sbg Proxy

*Karena malas berurusan dengan library datasets*

In [5]:
tydi_with_generated = pd.read_json("meta-llama_Llama-3.2-1B-Instruct-50_results.json")

### Mismatch antara jenis pertanyaan (dimana) dengan jenis jawaban (waktu)

In [10]:
# Definisikan kata-kata pertama yang ingin dicari
target_words = ['Dimana', 'dimanakah', 'Dimanakah', 'Dimankah', 'dimankah']

# Ambil subset data di mana question_text diawali dengan salah satu kata di target_words
filtered_data = [
    {"id": qid, "question_text": question, "answers": answers}
    for qid, question, answers in zip(dataset['train']['id'], dataset['train']['question_text'], dataset['train']['answers'])
    if question.split()[0] in target_words
]

# Konversi ke DataFrame untuk tampilan lebih mudah
filtered_df = pd.DataFrame(filtered_data)

In [15]:

# List ID yang ingin dicari
target_ids = [
    '496955121539262633-33', '-2609773390563272077-2', '-946255727274322126-4', 
    '-7749839855121808364-0', '4442256442092795257-8', '199208750651388504-2', 
    '6542172453366557069-7', '4919195816976139224-0', '1443260107176351473-1', 
    '6273515731581201009-9', '-7557435993194156840-18', '-5596213448756892216-7',  
    '-515183342052641630-0', '-4344805552053190465-2', '-1356951969628235591-3', 
    '132761720256571903-0', '733277053417590077-0', '8422091779176670015-4'
]

# Filter baris di mana kolom 'id' ada dalam target_ids
wrong_where_q = filtered_df[filtered_df['id'].isin(target_ids)]
num_wrong_where_q = wrong_where_q.shape[0]
print("Jumlah row yg pertanyaan 'Dimana' malah dijawab dengan jawaban waktu:", num_wrong_where_q)

percent_of_wrong_where_q = num_wrong_where_q * 100 / filtered_df.shape[0]
print("Persentase terhadap seluruh pertanyaan 'Dimana':", percent_of_wrong_where_q)

wrong_where_q.head()

Jumlah row yg pertanyaan 'Dimana' malah dijawab dengan jawaban waktu: 18
Persentase terhadap seluruh pertanyaan 'Dimana': 3.5502958579881656


,id,question_text,answers
0,496955121539262633-33,dimanakah Dr. Ernest François Eugène Douwes Dekker meninggal?,"{'text': ['28 Agustus 1950'], 'start_byte': [45], 'limit_byte': [60]}"
3,-2609773390563272077-2,Dimanakah Kucing Pallas pertama kali ditemukan ?,"{'text': ['2008'], 'start_byte': [961], 'limit_byte': [965]}"
24,-946255727274322126-4,"dimanakah Joseph Patrick ""Joe"" Kennedy dilahirkan?","{'text': ['1858'], 'start_byte': [318], 'limit_byte': [322]}"
28,-7749839855121808364-0,Dimanakah produksi film pertama dilakukan ?,"{'text': ['abad 20'], 'start_byte': [434], 'limit_byte': [441]}"
33,4442256442092795257-8,dimanakah Fiber optik pertama kali diciptakan?,"{'text': ['1970'], 'start_byte': [2952], 'limit_byte': [2956]}"
